# Face Re-ID

This notebook shows an example of training a face re-ID model (`FaceIdentifier`).

## Load Data
The data is from CelebA Dataset. For this Face re-ID task, only annotations of identities (`identity_CelebA.txt`) are needed. Each line of the annotations is `image_path identity` (e.g. `000001.jpg 2880`). I splited the data into 3 parts: training_set, val_set, and test_set after grouping them by identities. In every epoch, the model will be trained on training_set, and be monitored by computing validation loss on val_set.

I customed a class `IdentityDataset`, returning 3 images in tensor format, the first 2 images are from the same person, and the third is randomly chose from another person.

In [ ]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from matplotlib import pyplot as plt
from face_identifier.datasets import read_identities, IdentityDataset, split_train_valid_test


# paths
path = '/home/anthony/Documents/Homework Documents/Inno Lab Group Project/'
path_anno = path + 'identity_CelebA.txt'
# path_images = path + 'cropped'
path_images = path + 'img_align_celeba'

# read data
data = read_identities(path_anno, path_images, n_images=2)
training_index, validation_index, test_index = split_train_valid_test(data)
training_set = IdentityDataset(data, path_images, training_index, transform=None)
val_set = IdentityDataset(data, path_images, validation_index, transform=None)
test_set = IdentityDataset(data, path_images, test_index, transform=None)

# show images
training_set.draw(0)

## Build Model

CNN model is suitable in this feature extraction task. The core model `FaceIdentifier`, returning face ID as a 500 dims vector, is based on a powerful CNN ResNet50, with a extra full connecting network that includes a ReLU and a linear layer. A triplet model, wrapping the core model, is also implemented for training. It receives three images mentioned above as inputs, and returns 2 scalars representing the probabilities that image 2 and 3 are from the same person of image 1. The expecting label is `[1, 0]` because of the design of the dataset. 

I chose Adam optimizer and MSE loss function to train this model, because I want the prediction of the triplet model as close as `[1, 0]`.

During the training, the core model's parameters will be updated. Finally, all we need for generating face ID is the core model. The architecture is shown below:

![architecture](../docs/pictures/face-identifier.drawio.png)

In [ ]:
from face_identifier.models import FaceIdentifier, TripletModel


model = FaceIdentifier()
tri_model = TripletModel(model)
optimizer = torch.optim.Adam(tri_model.parameters(), lr=1e-3)
loss_func = torch.nn.MSELoss()
tri_model

## Train Model

During the training, the `ModelTrainer` will record the training and validation losses in every epoch. It will also save the state dict of the model with the lowest validation loss.

In [ ]:
import os
from face_identifier.processes import ModelTrainer, test


save_path = 'face-identifier-wrapped.pt'
if os.path.exists(save_path):
    tri_model.load_state_dict(torch.load(save_path))
trainer = ModelTrainer(tri_model, training_set, val_set, optimizer, loss_func, device=device, batch_size=30,
                       save_path=save_path, log_name='Id Loss')

In [ ]:
trainer.train(100)

According to the learning curve, overfitting occurred after the 68th epoch.
![learning curve](../docs/pictures/learning-curve.jpg)

In [ ]:
# load the best model
tri_model.load_state_dict(torch.load('face-identifier-wrapped.pt'))

# save the core model
torch.save(model.state_dict(), 'face-identifier.pt')

print(f'Best validation loss is {trainer.best_loss: .5f}')

## Evaluation & Visualization

First, we need to check the saved model.

In [ ]:
# load the best model
from face_identifier.models import FaceIdentifier, TripletModel
model = FaceIdentifier.load('face-identifier.pt')
tri_model = TripletModel(model)
loss_func = torch.nn.MSELoss()

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from face_identifier.processes import test, MODEL_TO_FUNC


test_loss = test(tri_model, tqdm(DataLoader(test_set, batch_size=30), 'Testing'), MODEL_TO_FUNC['identifier'], loss_func, device)
print(f'Test loss is {test_loss: .5f}')

The test loss roughly equals to the best validation loss, here we got the best model!!

### Evaluation
The core model can be evaluated by being wrapped as a binary classification model. The new model will predict whether 2 given images are from the same person. A special dataset, returning 2 images and a binary label, is also needed. The function `evaluate_classification` can handle these tasks.

In [ ]:
from face_identifier.evaluations import evaluate_classification


threshold = 0.8


model = FaceIdentifier.load('face-identifier.pt')
result = evaluate_classification(model, test_set, n_samples=100,
                                 threshold=threshold,
                                 batch_size=64, device=device)

print(f'Tested on {result["total"]} pairs of images.')
for key in ['accuracy', 'precision', 'recall', 'f1']:
    print(f'{key.title()}: {result[key]:.5%}')

### Visualization

You can also check the similarities of specific pictures. In this case, **similarity > 0.8** means two images are from the same person. The function `visualize` will use the vectors returned by the core model to compute the similarities of the given three images, and show the result.

In [ ]:
from face_identifier.evaluations import visualize

for i in range(5):
    visualize(model, test_set, i + 80)